In [1]:
import pandas as pd
import pandas_datareader.data as web
import pandas_datareader as pdreader
pdreader.famafrench.get_available_datasets()

/Users/markhendricks/Projects/finm-portfolio-2023/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


['F-F_Research_Data_Factors',
 'F-F_Research_Data_Factors_weekly',
 'F-F_Research_Data_Factors_daily',
 'F-F_Research_Data_5_Factors_2x3',
 'F-F_Research_Data_5_Factors_2x3_daily',
 'Portfolios_Formed_on_ME',
 'Portfolios_Formed_on_ME_Wout_Div',
 'Portfolios_Formed_on_ME_Daily',
 'Portfolios_Formed_on_BE-ME',
 'Portfolios_Formed_on_BE-ME_Wout_Div',
 'Portfolios_Formed_on_BE-ME_Daily',
 'Portfolios_Formed_on_OP',
 'Portfolios_Formed_on_OP_Wout_Div',
 'Portfolios_Formed_on_OP_Daily',
 'Portfolios_Formed_on_INV',
 'Portfolios_Formed_on_INV_Wout_Div',
 'Portfolios_Formed_on_INV_Daily',
 '6_Portfolios_2x3',
 '6_Portfolios_2x3_Wout_Div',
 '6_Portfolios_2x3_weekly',
 '6_Portfolios_2x3_daily',
 '25_Portfolios_5x5',
 '25_Portfolios_5x5_Wout_Div',
 '25_Portfolios_5x5_Daily',
 '100_Portfolios_10x10',
 '100_Portfolios_10x10_Wout_Div',
 '100_Portfolios_10x10_Daily',
 '6_Portfolios_ME_OP_2x3',
 '6_Portfolios_ME_OP_2x3_Wout_Div',
 '6_Portfolios_ME_OP_2x3_daily',
 '25_Portfolios_ME_OP_5x5',
 '25_Portf

In [2]:
startdate = '19800101'
enddate = '20231031'

In [3]:
info = pd.DataFrame(columns=['Name', 'Unit', 'Construction', 'Description'])
info.loc['MKT'] = ['Market', 'Excess Return', 'Market-cap-weighted', 'US Equities']
#info.loc['SMB'] = ['Size', 'Excess Return', 'Small Minus Big', 'Long small stocks and short big stocks']
info.loc['HML'] = ['Value', 'Excess Return', 'High Minus Low', 'Long value (high book-to-market) stocks and short growth (low book-to-market) stocks']
#info.loc['RMW'] = ['Profitability','Excess Return', 'Robust Minus Weak', 'Long profitability (income statement) and short low profitability']
#info.loc['CMA'] = ['Investment','Excess Return', 'Conservative Minus Agressive', 'Long stocks with low (conservative) investment (balance sheet) and short stocks with high (agressive) investment.']
info.loc['UMD'] = ['Momentum','Excess Return','Up Minus Down', 'Long stocks that have high recent returns and short stocks with low recent returns']
info.loc['RF'] = ['Risk-free rate', 'Total Return', 'Tbills', '']
info

,Name,Unit,Construction,Description
MKT,Market,Excess Return,Market-cap-weighted,US Equities
HML,Value,Excess Return,High Minus Low,Long value (high book-to-market) stocks and sh...
UMD,Momentum,Excess Return,Up Minus Down,Long stocks that have high recent returns and ...
RF,Risk-free rate,Total Return,Tbills,


In [4]:
rawdata = web.DataReader('F-F_Research_Data_5_Factors_2x3', data_source='famafrench',start=startdate,end=enddate)
facs = rawdata[0] / 100
# convert the yyyy-mm to timestamp object. default is to put first of month, but should be end of month
facs = facs.to_timestamp().resample('M').last()
facs.rename(columns={'Mkt-RF':'MKT'},inplace=True)

rawdata = web.DataReader('F-F_Momentum_Factor', data_source='famafrench',start=startdate,end=enddate)
umd = rawdata[0] / 100
# convert the yyyy-mm to timestamp object. default is to put first of month, but should be end of month
umd = umd.to_timestamp().resample('M').last()
umd.columns = ['UMD']

rf = facs[['RF']]
facs = facs.join(umd).drop(columns=['RF','RMW','SMB','CMA'])
facs

/var/folders/zx/3v_qt0957xzg3nqtnkv007d00000gn/T/ipykernel_52683/2959303215.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  rawdata = web.DataReader('F-F_Research_Data_5_Factors_2x3', data_source='famafrench',start=startdate,end=enddate)
/var/folders/zx/3v_qt0957xzg3nqtnkv007d00000gn/T/ipykernel_52683/2959303215.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  rawdata = web.DataReader('F-F_Research_Data_5_Factors_2x3', data_source='famafrench',start=startdate,end=enddate)
/var/folders/zx/3v_qt0957xzg3nqtnkv007d00000gn/T/ipykernel_52683/2959303215.py:7: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' in

,MKT,HML,UMD
Date,,,
1980-01-31,0.0551,0.0175,0.0755
1980-02-29,-0.0122,0.0061,0.0788
1980-03-31,-0.1290,-0.0101,-0.0955
1980-04-30,0.0397,0.0106,-0.0043
1980-05-31,0.0526,0.0038,-0.0112
...,...,...,...
2023-04-30,0.0061,-0.0004,0.0162
2023-05-31,0.0035,-0.0772,-0.0063
2023-06-30,0.0646,-0.0026,-0.0237


In [5]:
rawdata = web.DataReader('10_Industry_Portfolios', data_source='famafrench',start=startdate, end=enddate)
rets = rawdata[0] / 100
# convert the yyyy-mm to timestamp object. default is to put first of month, but should be end of month
rets = rets.to_timestamp().resample('M').last()
retsx = rets.subtract(rf['RF'],axis=0)
retsx

/var/folders/zx/3v_qt0957xzg3nqtnkv007d00000gn/T/ipykernel_52683/136190961.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  rawdata = web.DataReader('10_Industry_Portfolios', data_source='famafrench',start=startdate, end=enddate)
/var/folders/zx/3v_qt0957xzg3nqtnkv007d00000gn/T/ipykernel_52683/136190961.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  rawdata = web.DataReader('10_Industry_Portfolios', data_source='famafrench',start=startdate, end=enddate)
/var/folders/zx/3v_qt0957xzg3nqtnkv007d00000gn/T/ipykernel_52683/136190961.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other
Date,,,,,,,,,,
1980-01-31,0.0232,0.0829,0.0627,0.1342,0.0656,-0.0286,0.0051,-0.0209,0.0076,0.0474
1980-02-29,-0.0574,-0.0719,-0.0339,0.1205,-0.0444,-0.0237,-0.0574,-0.0552,-0.0428,-0.0428
1980-03-31,-0.0934,-0.1000,-0.1297,-0.1910,-0.1387,-0.0383,-0.0985,-0.0403,-0.0780,-0.1348
1980-04-30,0.0523,-0.0303,0.0195,0.0560,-0.0054,0.0606,0.0210,0.0303,0.1074,0.0480
1980-05-31,0.0713,0.0352,0.0542,0.0328,0.0401,0.0419,0.0992,0.0615,0.0378,0.0731
...,...,...,...,...,...,...,...,...,...,...
2023-04-30,0.0281,-0.1559,-0.0093,0.0224,-0.0009,0.0064,0.0184,0.0377,0.0137,0.0148
2023-05-31,-0.0571,0.1316,-0.0590,-0.0987,0.0787,-0.0976,0.0026,-0.0403,-0.0618,-0.0301
2023-06-30,0.0244,0.2426,0.1029,0.0602,0.0551,0.0423,0.0749,0.0426,0.0221,0.0704


In [6]:
with pd.ExcelWriter('../data/midterm_2_data.xlsx') as writer:  
    info.to_excel(writer, sheet_name= 'descriptions')
    facs.to_excel(writer, sheet_name='factors (excess returns)')
    retsx.to_excel(writer, sheet_name='portfolios (excess returns)')
    rf.to_excel(writer, sheet_name='risk-free rate')